# Data Cleaning

#### To-do:

- drop irrelevant columns
- deal with nulls
- string cleaning
- explore combining target classes down to two classes


In [5]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
%matplotlib inline
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import normalize

In [6]:
df_raw = pd.read_csv('../data/raw/raw_training_set_full.csv')

### Drop irrelevant columns